# Backpropagation for Regression – Annotated Notebook
This notebook demonstrates how to implement manual backpropagation for a simple regression problem using NumPy. We will:
- Generate synthetic data
- Initialize parameters
- Perform forward and backward passes
- Update weights manually
- Train the network and predict values

Created by-Vipul Ingale


## Step 1 – Import Libraries
We import:
- **NumPy** for numerical computations
- **Pandas** for tabular data handling
- **warnings** to suppress library warnings for cleaner output

In [16]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

## Step 2 – Create a Synthetic Dataset
We generate:
- 20 random samples
- **CGPA** between 5.0 and 10.0 (2 decimals)
- **Profile score** between 6 and 15 (integers)
- **LPA** = `0.5 * CGPA + 0.3 * Profile_score + noise` (noise from normal distribution)
Finally, store the data in a Pandas DataFrame.

In [17]:
np.random.seed(42)
num_samples = 20
cgpa = np.round(np.random.uniform(5.0, 10.0, num_samples), 2)
profile_score = np.random.randint(6, 16, num_samples)
noise = np.round(np.random.normal(0, 0.5, num_samples), 2)
lpa = np.round(0.5 * cgpa + 0.3 * profile_score + noise, 2)
df = pd.DataFrame({
    'cgpa': cgpa,
    'profile_score': profile_score,
    'lpa': lpa
})
print(df)

    cgpa  profile_score   lpa
0   6.87             15  7.56
1   9.75              8  6.89
2   8.66             12  8.40
3   7.99              9  7.48
4   5.78             14  6.91
5   5.78              8  5.48
6   5.29             10  5.04
7   9.33              8  6.86
8   8.01             12  7.38
9   8.54             10  8.03
10  5.10             14  6.91
11  9.85             12  7.82
12  9.16              7  5.59
13  6.06              9  5.21
14  5.91             14  7.24
15  5.92              7  5.22
16  6.52             15  8.13
17  7.62             14  7.09
18  7.16             15  8.36
19  6.46             10  6.24


## Step 3 – Parameter Initialization
Function to initialize neural network parameters given layer dimensions (`layer_dims`).
- Weights `W` start at 0.1
- Biases `b` start at 0
- Returns a dictionary of parameters

In [3]:
def initialize_parameters(layer_dims):
    np.random.seed(3)
    parameters = {}
    L = len(layer_dims)
    for l in range(1, L):
        parameters['W' + str(l)] = np.ones((layer_dims[l-1], layer_dims[l])) * 0.1
        parameters['b' + str(l)] = np.zeros((layer_dims[l], 1))
    return parameters

### Example: Initializing for [2,2,1]
This means:
- 2 input neurons
- 2 hidden neurons
- 1 output neuron


for understanding purpose,we are creating NN with only 1 hidden layer
- parameter W1 represents [[W11,W12],[W21,W22]]
- parameter b1 represents bais for hidden layer and b2 for output layer

In [19]:
initialize_parameters([2,2,1])

{'W1': array([[0.1, 0.1],
        [0.1, 0.1]]),
 'b1': array([[0.],
        [0.]]),
 'W2': array([[0.1],
        [0.1]]),
 'b2': array([[0.]])}

## Step 4 – Linear Function
Computes:
$$
Z = W^T A_{prev} + b
$$

- A_prev is basically output from prev layer

In [21]:
def Linear_func(A_prev, W, b):
    Z = np.dot(W.T, A_prev) + b
    return Z

## Step 5 – Forward Pass
Iteratively applies the linear function for each layer.
Returns the final output and the activations from the last hidden layer.

In [22]:
def forward_pass(X, parameters):
    A = X
    L = len(parameters) // 2
    for l in range(1, L+1):
        A_prev = A
        Wl = parameters['W' + str(l)]
        bl = parameters['b' + str(l)]
        A = Linear_func(A_prev, Wl, bl)
    return A, A_prev

## Step 6 – Single Example Forward Pass
We take the first row from the dataset and make a forward pass with initial parameters.

In [23]:
X = df[['cgpa', 'profile_score']].values[0].reshape(2,1)
y = df[['lpa']].values[0][0]
parameters = initialize_parameters([2,2,1])
y_hat, A1 = forward_pass(X, parameters)

## Step 7 – Flatten Prediction
Get a scalar value instead of a nested array.

In [24]:
y_hat = y_hat[0][0]

## Step 8 – View Hidden Layer Output

In [25]:
A1

array([[2.187],
       [2.187]])

## Step 9 – Compare Actual vs Predicted

In [26]:
y, y_hat

(np.float64(7.56), np.float64(0.43740000000000007))

## Step 10 – Inspect Parameters

In [27]:
parameters

{'W1': array([[0.1, 0.1],
        [0.1, 0.1]]),
 'b1': array([[0.],
        [0.]]),
 'W2': array([[0.1],
        [0.1]]),
 'b2': array([[0.]])}

## Step 11 – Update Parameters (Manual Backpropagation)
Uses derivatives of MSE loss to update the weights manually.

In [28]:
def update_parameters(parameters, y, y_hat, A1, X, L):
    parameters['W2'][0][0] += (L * 2 * (y - y_hat) * A1[0][0])
    parameters['W2'][1][0] += (L * 2 * (y - y_hat) * A1[1][0])
    parameters['b2'][0][0] += (L * 2 * (y - y_hat))

    parameters['W1'][0][0] += (L * 2 * (y - y_hat) * parameters['W2'][0][0] * X[0][0])
    parameters['W1'][0][1] += (L * 2 * (y - y_hat) * parameters['W2'][0][0] * X[1][0])
    parameters['b1'][0][0] += (L * 2 * (y - y_hat) * parameters['W2'][0][0])

    parameters['W1'][1][0] += (L * 2 * (y - y_hat) * parameters['W2'][1][0] * X[0][0])
    parameters['W1'][1][1] += (L * 2 * (y - y_hat) * parameters['W2'][1][0] * X[1][0])
    parameters['b1'][1][0] += (L * 2 * (y - y_hat) * parameters['W2'][1][0])

## Step 12 – Training Loop
Randomly selects samples each iteration, runs forward pass, updates weights, and prints loss per epoch.

In [29]:
def backpropagation_train(epochs=30, parms=[2,2,1], learning_rate=0.001):
    parameters = initialize_parameters(parms)
    for i in range(epochs):
        Loss = []
        for j in range(df.shape[0]):
            random_ip = df.sample()
            X = random_ip[['cgpa', 'profile_score']].values.reshape(2,1)
            y = random_ip['lpa'].values.reshape(1,1)
            y_hat, A1 = forward_pass(X, parameters)
            y_hat = y_hat[0][0]
            update_parameters(parameters, y, y_hat, A1, X, learning_rate)
            Loss.append((y - y_hat)**2)
        print('Epoch -', i+1, 'Loss -', np.array(Loss).mean())

backpropagation_train()

Epoch - 1 Loss - 13.328097973710246
Epoch - 2 Loss - 0.19289568020544223
Epoch - 3 Loss - 0.4411186461839799
Epoch - 4 Loss - 0.40780316581811593
Epoch - 5 Loss - 0.21300332646743594
Epoch - 6 Loss - 0.302362853773904
Epoch - 7 Loss - 0.2488150936830443
Epoch - 8 Loss - 0.3740165476995419
Epoch - 9 Loss - 0.31586229369334773
Epoch - 10 Loss - 0.5071108860392458
Epoch - 11 Loss - 0.5262693479836422
Epoch - 12 Loss - 0.4288874502614931
Epoch - 13 Loss - 0.22274858443195567
Epoch - 14 Loss - 0.2589400435100473
Epoch - 15 Loss - 0.49958209879101806
Epoch - 16 Loss - 0.37675618399297356
Epoch - 17 Loss - 0.40754506345339864
Epoch - 18 Loss - 0.3744376761838866
Epoch - 19 Loss - 0.40344540203633994
Epoch - 20 Loss - 0.40438064164634835
Epoch - 21 Loss - 0.33650086212018526
Epoch - 22 Loss - 0.40220069086189075
Epoch - 23 Loss - 0.3270757257127329
Epoch - 24 Loss - 0.5394271164458627
Epoch - 25 Loss - 0.26723452303747774
Epoch - 26 Loss - 0.37261375754245096
Epoch - 27 Loss - 0.48189089831270

## Step 13 – Prediction Function
Takes new CGPA, Profile Score, and actual LPA to predict and compute error.

In [30]:
def predict(x1, x2, y):
    X = np.array([x1, x2]).reshape(2,1)
    y = np.array([y]).reshape(1,1)
    Predicted, _ = forward_pass(X, parameters)
    error = Predicted - y
    return Predicted, error

## Step 14 – Test Prediction

In [31]:
predict(9, 8, 7)

(array([[0.34]]), array([[-6.66]]))